In [1]:
from collections import defaultdict
from lmfdb import db
import joblib

INFO:LMFDB@2025-10-04 10:49:52,846: Configuration = {'flask_options': {'port': 37777, 'host': '127.0.0.1', 'debug': True, 'use_reloader': False}, 'postgresql_options': {'port': 5432, 'host': 'devmirror.lmfdb.xyz', 'dbname': 'lmfdb', 'user': 'lmfdb', 'password': '****'}, 'logging_options': {'logfile': 'flasklog', 'slowcutoff': 0.1, 'slowlogfile': 'slow_queries.log', 'editor': '', 'loglevel': 20}} 
INFO:LMFDB@2025-10-04 10:49:52,978: Connecting to PostgresSQL server as: user=lmfdb host=devmirror.lmfdb.xyz port=5432 dbname=lmfdb... 
INFO:LMFDB@2025-10-04 10:49:54,707: Done!
 connection = <connection object at 0x7de253f42200; dsn: 'user=lmfdb password=xxx dbname=lmfdb host=devmirror.lmfdb.xyz port=5432', closed: 0> 
2025-10-04 10:49:56,059 - SELECT pg_is_in_recovery() ran in  0.2722287178039551s 
2025-10-04 10:49:56,867 - SELECT current_setting('is_superuser') ran in  0.5404033660888672s 
INFO:LMFDB@2025-10-04 10:49:57,135: User: lmfdb 
INFO:LMFDB@2025-10-04 10:49:57,136: Read only: True 


# Downloading Data

In [ ]:
query = [('ec_torsion_growth', {'degree' : int(2)})]
torsion_growth_with_local_data = db.ec_torsion_growth.join_search(query=query, 
                                        projection=["lmfdb_label", "torsion", "degree", "field", "conductor", ("ec_curvedata", "torsion_structure"),("ec_curvedata", "absD"),("ec_curvedata", "ainvs"),("ec_localdata", "reduction_type"),("ec_localdata","prime"),("ec_localdata","kodaira_symbol")], 
                                        join=[("ec_torsion_growth", "lmfdb_label", "ec_curvedata","lmfdb_label"), ("ec_torsion_growth", "lmfdb_label", "ec_localdata","lmfdb_label")])


In [ ]:
# Before running this make sure having a good internet connection.
# It will take 2 to 4 hours

In [ ]:
torsion_growth_with_local_data = list(torsion_growth_with_local_data)

In [ ]:
x = polygen(ZZ,'x')

for elliptic_curve in torsion_growth_with_local_data:
    elliptic_curve['torsion_before'] = elliptic_curve[('ec_curvedata','torsion_structure')]
    elliptic_curve['torsion_after'] = elliptic_curve['torsion']
    elliptic_curve['curve_discriminant'] = elliptic_curve[('ec_curvedata','absD')]
    elliptic_curve['prime'] = elliptic_curve[('ec_localdata', 'prime')]
    elliptic_curve['kodaira_symbol'] = elliptic_curve[('ec_localdata', 'kodaira_symbol')]
    elliptic_curve['reduction_type'] = elliptic_curve[('ec_localdata','reduction_type')]
    elliptic_curve['coefficients'] = elliptic_curve[("ec_curvedata", "ainvs")]
    
    del elliptic_curve['torsion']
    del elliptic_curve[('ec_curvedata','torsion_structure')]
    del elliptic_curve[('ec_curvedata','absD')]
    del elliptic_curve[('ec_localdata', 'prime')]
    del elliptic_curve[('ec_localdata', 'kodaira_symbol')]
    del elliptic_curve[('ec_localdata','reduction_type')]
    del elliptic_curve[("ec_curvedata", "ainvs")]
    
    f = elliptic_curve['field']
    K.<a> = NumberField(f[0]*x^2+ f[1]*x + f[2]) 
    elliptic_curve['disc_abs'] = K.absolute_discriminant()

In [ ]:
# Diccionario para agrupar por lmfdb_label, torsion_before y torsion_after
grouped_data = defaultdict(lambda: {'lmfdb_label': '', 'coefficients': [],
      'degree': None, 'field': [], 'conductor': None, 'torsion_before': [], 'torsion_after': [], 'curve_discriminant': None, 'disc_abs': None, 'prime_red_kodaira': []})

# Agrupamos los datos
for entry in torsion_growth_with_local_data:
    key = (entry['lmfdb_label'], tuple(entry['torsion_before']), tuple(entry['torsion_after']),int(entry['disc_abs']))
    grouped_data[key]['lmfdb_label'] = entry['lmfdb_label']
    grouped_data[key]['degree'] = entry['degree']
    grouped_data[key]['field'] = entry['field']
    grouped_data[key]['conductor'] = entry['conductor']
    grouped_data[key]['torsion_before'] = entry['torsion_before']
    grouped_data[key]['torsion_after'] = entry['torsion_after']
    grouped_data[key]['curve_discriminant'] = entry['curve_discriminant']
    grouped_data[key]['disc_abs'] = entry['disc_abs']
    grouped_data[key]['coefficients'] = entry['coefficients']
    
    # Añadimos el triple (prime, kodaira_symbol, reduction_type)
    grouped_data[key]['prime_red_kodaira'].append((entry['prime'],  entry['reduction_type'],entry['kodaira_symbol']))

# Convertimos el resultado a una lista de diccionarios
torsion_growth_with_local_data = list(grouped_data.values())

In [ ]:
torsion_growth_with_local_data[0]

In [ ]:
import joblib

# Definir el tamaño del fragmento
tamano_fragmento = 100000  

# Dividir y guardar cada fragmento
for i in range(0, len(torsion_growth_with_local_data), tamano_fragmento):
    fragmento = torsion_growth_with_local_data[i:i + tamano_fragmento]
    nombre_archivo = f'torsion_growth_with_local_data_{i // tamano_fragmento}.pkl'
    joblib.dump(fragmento, nombre_archivo)
    print(f'Fragment {i // tamano_fragmento} saved as {nombre_archivo}')



# Loading Data

In [2]:
import glob

# List to store the loaded fragments
torsion_growth_with_local_data = []

# Load each fragment and add it to the complete list
for nombre_archivo in sorted(glob.glob('data/torsion_growth_with_local_data_*.pkl')):
    fragmento = joblib.load(nombre_archivo)
    torsion_growth_with_local_data.extend(fragmento)

print("List of curves ready.")


List of curves ready.


In [3]:
torsion_growth_with_local_data_grouped_by_label= {}

# List to store the loaded fragments
for curve in torsion_growth_with_local_data:
    label = curve['lmfdb_label']
    # Load each fragment and add it to the complete list
    if label not in torsion_growth_with_local_data_grouped_by_label:
        torsion_growth_with_local_data_grouped_by_label[label] = []
    torsion_growth_with_local_data_grouped_by_label[label].append(curve)

In [4]:
torsion_growth_with_local_data[0]

{'lmfdb_label': '100.a1',
 'coefficients': [0, -1, 0, -1033, -12438],
 'degree': 2,
 'field': [-1, -1, 1],
 'conductor': 100,
 'torsion_before': [2],
 'torsion_after': [2, 2],
 'curve_discriminant': 31250000,
 'disc_abs': 5,
 'prime_red_kodaira': [(2, 0, 4), (5, 0, -7)]}

In [5]:
torsion_growth_with_local_data_grouped_by_label['100.a1']

[{'lmfdb_label': '100.a1',
  'coefficients': [0, -1, 0, -1033, -12438],
  'degree': 2,
  'field': [-1, -1, 1],
  'conductor': 100,
  'torsion_before': [2],
  'torsion_after': [2, 2],
  'curve_discriminant': 31250000,
  'disc_abs': 5,
  'prime_red_kodaira': [(2, 0, 4), (5, 0, -7)]},
 {'lmfdb_label': '100.a1',
  'coefficients': [0, -1, 0, -1033, -12438],
  'degree': 2,
  'field': [4, -1, 1],
  'conductor': 100,
  'torsion_before': [2],
  'torsion_after': [6],
  'curve_discriminant': 31250000,
  'disc_abs': -15,
  'prime_red_kodaira': [(2, 0, 4), (5, 0, -7)]}]

# Auxiliar Functions

In [6]:

def explicit_torsion(torsion_before, torsion_after, curve):
    return curve['torsion_before'] == torsion_before and curve['torsion_after'] == torsion_after

def add_N_torsion_point(order,curve):
    
    torsion_after = curve['torsion_after']
    torsion_before = curve['torsion_before']
    
    length_before = len(torsion_before)
    length_after = len(torsion_after)
    
    add_torsion = False
    
    if length_after == length_before:
        for i in range(length_before):
            add_torsion = add_torsion or ((torsion_after[i]%order == 0) and (torsion_before[i]%order != 0))
        
        
        return add_torsion
    elif not torsion_before:
        return any([torsion_after[i]%order == 0 for i in range(length_after)])
    
    else:
        return (torsion_after[1]%order == 0) or ((torsion_after[0]%order == 0) and (torsion_before[0]%order != 0))
            
    
def addditiveReduction(prime, curve):
    return any([p==prime and r == 0 for p,r,k in curve['prime_red_kodaira']])

def kodairaSymbol(prime, symbol, curve):
    return any([p==prime and k==symbol for p,r,k in curve['prime_red_kodaira']])

def extractExponent(number, prime):
    return [e for p,e in list(factor(number)) if p == 3][0]

def removeDuplicatesByProperty(curves, propName, propValue):
    return [curve for curve in curves if curve[propName] == propValue]

def compute_c6(curve):
    a1,a2,a3,a4,a6 = curve['coefficients']
    b2 = a1^2+4*a2 
    b4 = 2*a4 + a1*a3
    b6 = a3^2+4*a6
    return -b2^3+36*b2*b4-216*b6

def compute_c4(curve):
    a1,a2,a3,a4,a6 = curve['coefficients']
    b2 = a1^2+4*a2 
    b4 = 2*a4 + a1*a3
    return b2^2-24*b4


In [7]:
add_N_torsion_point(4, torsion_growth_with_local_data[0])

False

# Conjectures case by case

### $\mathcal{C}_1\longrightarrow \mathcal{C}_3$

In [8]:
curves_C1_C3 = list(filter(lambda curve : explicit_torsion([], [3],curve), torsion_growth_with_local_data))
len(curves_C1_C3)

141560

# Conjecture

If a prime \( p \) divides \( D \), the discriminant of $\mathbb{Q}(\sqrt{d})$, then the curve has additive bad reduction at \( p \), unless \( p = 3 \), where $\mathbb{Q}(\sqrt{d})$ is the field where the torsion grows.

If \( p = 3 \), there are examples with all types of reduction at 3, although it might be possible to relate the behavior of 3, i.e., whether it appears or not, to its type of reduction.


In [9]:
def direct_conjecture_C1_C3(curve):
    primes_discriminant = [p for p,e in list(curve['disc_abs'].factor())]
    bad_reduction_primes = dict()
    for p, red, kodaira in curve['prime_red_kodaira']:
        bad_reduction_primes[p] = red
    for p in primes_discriminant:
        if p not in bad_reduction_primes.keys():
            bad_reduction_primes[p] = "good reduction"
        
    return all([(bad_reduction_primes[p] == 0) or (p ==3 )
                for p in primes_discriminant])

len([curve for curve in curves_C1_C3 if not direct_conjecture_C1_C3(curve)])

0

In [10]:
def reciprocal_conjecture_C1_C3(curve):
    
    primes_with_additive_reduction = [(p, kodaira) for p,red,kodaira in curve['prime_red_kodaira'] if red == 0]
    conjecture_without_3 = all([(curve['disc_abs']%prime ==0) or kodaira ==-4 or kodaira == 4 for (prime, kodaira) in primes_with_additive_reduction if prime!=3])
    
    return all([conjecture_without_3])


len([curve for curve in curves_C1_C3 if  not reciprocal_conjecture_C1_C3(curve)])

0

In [11]:
set([((compute_c4(curve)).valuation(3),(compute_c6(curve)).valuation(3), (curve['curve_discriminant']).valuation(3))  for curve in curves_C1_C3 if   (kodairaSymbol(3,-7, curve)) and curve['disc_abs']%3!=0])

{(2, 3, 9)}

In [12]:
set([((compute_c4(curve)).valuation(3),(compute_c6(curve)).valuation(3), (curve['curve_discriminant']).valuation(3))  for curve in curves_C1_C3 if   (kodairaSymbol(3,-7, curve)) and curve['disc_abs']%3==0])

{(2, 3, 9)}

In [13]:
len([curve for curve in curves_C1_C3 if 
     (kodairaSymbol(3,-1, curve)) and curve['disc_abs']%3==0])

5257

In [14]:
set([extractExponent(curve['curve_discriminant'], 3) for curve in curves_C1_C3 if 
     (kodairaSymbol(3,4, curve)) and curve['disc_abs']%3==0])

set()

In [15]:
curves_C1_C3 = list(filter(lambda curve : explicit_torsion([], [3],curve), torsion_growth_with_local_data))
len(curves_C1_C3)

141560

### $\mathcal{C}_1\longrightarrow \mathcal{C}_5$

In [16]:
curves_C1_C5 = list(filter(lambda curve : explicit_torsion([], [5],curve), torsion_growth_with_local_data))

In [17]:
curves_C1_C5[567]

{'lmfdb_label': '140581.a2',
 'coefficients': [0, 1, 1, -54406, -4892226],
 'degree': 2,
 'field': [2, -1, 1],
 'conductor': 140581,
 'torsion_before': [],
 'torsion_after': [5],
 'curve_discriminant': 43987896258901,
 'disc_abs': -7,
 'prime_red_kodaira': [(19, -1, 9), (7, 0, -1), (151, 1, 5)]}

In [18]:
len(curves_C1_C5)

4676

  # First Conjecture: 

If a prime $ p $ divides $ D $, the discriminant of $ \mathbb{Q}(\sqrt{d}) $, then the curve has additive bad reduction at $ p $, 

where $ \mathbb{Q}(\sqrt{d}) $ is the field where the torsion grows.



In [19]:
def direct_conjecture_C1_C5(curve):
    primes_discriminant = [p for p,e in list(curve['disc_abs'].factor())]
    bad_reduction_primes = dict()
    for p, red, kodaira in curve['prime_red_kodaira']:
        bad_reduction_primes[p] = red
    for p in primes_discriminant:
        if p not in bad_reduction_primes.keys():
            bad_reduction_primes[p] = "good reduction"
        
    return all([(bad_reduction_primes[p] == 0) for p in primes_discriminant])


len([curve for curve in curves_C1_C5 if not direct_conjecture_C1_C5(curve)])

0

# Second Conjecture: 

If the curve has additive reduction at $ p \neq 5 $, then $ p $ divides $ D $, the discriminant of $ \mathbb{Q}(\sqrt{d}) $. 

If the curve has additive reduction at 5, then $ 5 \mid d $ if, and only if, the Kodaira symbol at 5 is neither II nor III. 

where $ \mathbb{Q}(\sqrt{d}) $ is the field where the torsion grows.



In [20]:
def reciprocal_conjecture_C1_C5(curve):
    primes_with_additive_reduction = [(p, kodaira) for p,red,kodaira in curve['prime_red_kodaira'] if red == 0 ]
    conjecture_without_5 = all([(curve['disc_abs']%prime ==0) for (prime, kodaira) in primes_with_additive_reduction if prime !=5])
    conjecture_for_5_reciprocal = all([curve['disc_abs']%prime ==0 for prime,kodaira in primes_with_additive_reduction if not (kodaira in [2,3]) and prime== 5])
    conjecture_for_5_direct = all([curve['disc_abs']%prime !=0 for prime,kodaira in primes_with_additive_reduction if (kodaira in [2,3]) and prime== 5])
    return all([conjecture_without_5,conjecture_for_5_direct, conjecture_for_5_reciprocal])



In [21]:
counterexamples = [curve for curve in curves_C1_C5 if not reciprocal_conjecture_C1_C5(curve)]
len(counterexamples)

0

In [22]:
[curve for curve in curves_C1_C5 if curve['disc_abs'] == 5][4]

{'lmfdb_label': '101075.g1',
 'coefficients': [0, 1, 1, -29758, 1967019],
 'degree': 2,
 'field': [-1, -1, 1],
 'conductor': 101075,
 'torsion_before': [],
 'torsion_after': [5],
 'curve_discriminant': 1804251921875,
 'disc_abs': 5,
 'prime_red_kodaira': [(5, 0, -1), (311, 1, 5), (13, -1, 9)]}

### $\mathcal{C}_1\longrightarrow \mathcal{C}_7$

In [23]:
curves_C1_C7 = list(filter(lambda curve : explicit_torsion([], [7],curve), torsion_growth_with_local_data))

In [24]:
len(curves_C1_C7)

606

# First Conjecture: 

If a prime $ p $ divides $ D $, the discriminant of $ \mathbb{Q}(\sqrt{d}) $, then the curve has additive bad reduction at $ p $, 

where $ \mathbb{Q}(\sqrt{d}) $ is the field where the torsion grows.


In [25]:
def direct_conjecture_C1_C7(curve):
    primes_discriminant = [p for p,e in list(curve['disc_abs'].factor())]
    bad_reduction_primes = dict()
    for p, red, kodaira in curve['prime_red_kodaira']:
        bad_reduction_primes[p] = red
    for p in primes_discriminant:
        if p not in bad_reduction_primes.keys():
            bad_reduction_primes[p] = "good reduction"
        
    return all([(bad_reduction_primes[p] == 0) for p in primes_discriminant])



len([curve for curve in curves_C1_C7 if not direct_conjecture_C1_C7(curve)])

0

# Second Conjecture: 

If the curve has additive reduction at $ p \neq 7 $, then $ p $ divides $ D $, the discriminant of $ \mathbb{Q}(\sqrt{d}) $. 

If the curve has additive reduction at 7, then $ 7 \mid d $ if, and only if, the Kodaira symbol at 7 is not II. 

where $ \mathbb{Q}(\sqrt{d}) $ is the field where the torsion grows.


In [26]:
def reciprocal_conjecture_C1_C7(curve):
    
    primes_with_additive_reduction = [(p, kodaira) for p,red,kodaira in curve['prime_red_kodaira'] if red == 0 ]
    conjecture_without_7 = all([(curve['disc_abs']%prime ==0) for (prime, kodaira) in primes_with_additive_reduction if prime !=7])
    conjecture_for_7_reciprocal = all([curve['disc_abs']%prime ==0 for prime,kodaira in primes_with_additive_reduction if not (kodaira in [2]) and prime== 7])
    conjecture_for_7_direct = all([curve['disc_abs']%prime !=0 for prime,kodaira in primes_with_additive_reduction if (kodaira in [2]) and prime== 7])
    
    
    return all([conjecture_without_7,conjecture_for_7_direct, conjecture_for_7_reciprocal])


In [27]:
counterexamples = [curve for curve in curves_C1_C7 if not reciprocal_conjecture_C1_C7(curve)]
len(counterexamples)

0

### $\mathcal{C}_1\longrightarrow \mathcal{C}_9$

In [28]:
curves_C1_C9 = list(filter(lambda curve : explicit_torsion([], [9],curve), torsion_growth_with_local_data))

In [29]:
len(curves_C1_C9)

225

# First Conjecture: 

If a prime $ p $ divides $ D $, the discriminant of $ \mathbb{Q}(\sqrt{d}) $, then the curve has additive bad reduction at $ p $, 

where $ \mathbb{Q}(\sqrt{d}) $ is the field where the torsion grows.
 

In [30]:
def direct_conjecture_C1_C9(curve):
    primes_discriminant = [p for p,e in list(curve['disc_abs'].factor())]
    bad_reduction_primes = dict()
    for p, red, kodaira in curve['prime_red_kodaira']:
        bad_reduction_primes[p] = red
    for p in primes_discriminant:
        if p not in bad_reduction_primes.keys():
            bad_reduction_primes[p] = "good reduction"
        
    return all([(bad_reduction_primes[p] == 0) for p in primes_discriminant])



len([curve for curve in curves_C1_C9 if not direct_conjecture_C1_C9(curve)])

0

# Second Conjecture: 

If the curve has additive reduction at \( p \neq 3 \), then $ p \mid D $, the discriminant of $ \mathbb{Q}(\sqrt{d}) $.

If the curve has additive reduction at 3, then $ 3 \mid d $ if, and only if, the Kodaira symbol at 3 is not IV.

where $ \mathbb{Q}(\sqrt{d}) $ is the field where the torsion grows.


In [31]:
def reciprocal_conjecture_C1_C9(curve):
    
    primes_with_additive_reduction = [(p, kodaira) for p,red,kodaira in curve['prime_red_kodaira'] if red == 0 ]
    conjecture_without_9 = all([(curve['disc_abs']%prime ==0) for (prime, kodaira) in primes_with_additive_reduction if prime !=3])
    conjecture_for_9_reciprocal = all([curve['disc_abs']%prime ==0 for prime,kodaira in primes_with_additive_reduction if not (kodaira in [4]) and prime== 3])
    conjecture_for_9_direct = all([curve['disc_abs']%prime !=0 for prime,kodaira in primes_with_additive_reduction if (kodaira in [4]) and prime== 3])
    
    
    return all([conjecture_without_9,conjecture_for_9_direct, conjecture_for_9_reciprocal])


In [32]:
counterexamples = [curve for curve in curves_C1_C9 if not reciprocal_conjecture_C1_C9(curve)]
len(counterexamples)

0

In [33]:
counterexamples = []
for curve in curves_C1_C9:
    for p,r,k in curve['prime_red_kodaira']:
        if p == 3 and r!=0:
            counterexamples.append(curve)

len(counterexamples)
    

58

In [34]:
counterexamples[0]

{'lmfdb_label': '10878.bg2',
 'coefficients': [1, 1, 1, -233584, -43948591],
 'degree': 2,
 'field': [2, -1, 1],
 'conductor': 10878,
 'torsion_before': [],
 'torsion_after': [9],
 'curve_discriminant': 15046841202430464,
 'disc_abs': -7,
 'prime_red_kodaira': [(3, -1, 13), (37, 1, 5), (7, 0, -7), (2, 1, 13)]}

### $\mathcal{C}_2\longrightarrow \mathcal{C}_4$

In [35]:
curves_C2_C4 = list(filter(lambda curve : explicit_torsion([2], [4],curve), torsion_growth_with_local_data))

In [36]:
len(curves_C2_C4)

425724

In [37]:
def reciprocal_conjecture_C2_C4(curve):
    label = curve['lmfdb_label']
    curves_with_fixed_label = list(filter(lambda curve : explicit_torsion([2], [4],curve), torsion_growth_with_local_data_grouped_by_label[label]))
    discriminants = [ec['disc_abs'] for ec in curves_with_fixed_label]
    prop = True
    for p, red, kodaira in curve['prime_red_kodaira']:
        if (red == 0) and (p!=2):
            prop = prop and any([discriminant%p == 0 for discriminant in discriminants])        
    return prop

In [38]:
counterexamples = [curve for curve in curves_C2_C4 if not reciprocal_conjecture_C2_C4(curve)]
len(counterexamples)

619

In [39]:
curve = counterexamples[0]
curve

{'lmfdb_label': '100800.ea8',
 'coefficients': [0, 0, 0, 27683700, -35589962000],
 'degree': 2,
 'field': [-6, 0, 1],
 'conductor': 100800,
 'torsion_before': [2],
 'torsion_after': [4],
 'curve_discriminant': 1905042624694272000000000,
 'disc_abs': 24,
 'prime_red_kodaira': [(2, 0, -14), (5, 0, -7), (7, -1, 8), (3, 0, -16)]}

### $\mathcal{C}_4\longrightarrow \mathcal{C}_8$

In [40]:
def reciprocal_conjecture_C4_C8(curve):
    label = curve['lmfdb_label']
    curves_with_fixed_label = list(filter(lambda curve : explicit_torsion([4], [8],curve), torsion_growth_with_local_data_grouped_by_label[label]))
    discriminants = [ec['disc_abs'] for ec in curves_with_fixed_label]
    prop = True
    for p, red, kodaira in curve['prime_red_kodaira']:
        if (red == 0) and (p!=2):
            prop = prop and any([discriminant%p == 0 for discriminant in discriminants])        
    return prop

In [41]:
curves_C4_C8 = list(filter(lambda curve : explicit_torsion([4], [8],curve), torsion_growth_with_local_data))
counterexamples = [curve for curve in curves_C4_C8 if not reciprocal_conjecture_C4_C8(curve)]
len(counterexamples)

0

### $\mathcal{C}_8\longrightarrow \mathcal{C}_{16}$

In [42]:
def reductionType(type_red, curve):
    answer = False
    for a,b,c in curve['prime_red_kodaira']:
        answer = ((b == type_red) and (a != 2)) or answer
    return answer
    

In [43]:
curves_C8_C16 = list(filter(lambda curve : explicit_torsion([8], [16],curve), torsion_growth_with_local_data))
counterexamples = [curve for curve in curves_C1_C5 if reductionType(0,curve)]
len(counterexamples)
counterexamples[0]

{'lmfdb_label': '100050.k2',
 'coefficients': [1, 1, 0, 1067425, -3165802875],
 'degree': 2,
 'field': [-1, -1, 1],
 'conductor': 100050,
 'torsion_before': [],
 'torsion_after': [5],
 'curve_discriminant': 4408691566561200000000,
 'disc_abs': 5,
 'prime_red_kodaira': [(3, -1, 14),
  (29, -1, 5),
  (23, -1, 9),
  (5, 0, -4),
  (2, -1, 14)]}

### $\mathcal{C}_8\longrightarrow \mathcal{C}_2\times\mathcal{C}_{8}$

# Torsion after has a new point of order $4$

In [44]:
curves_add_4_torsion = list(filter(lambda curve : add_N_torsion_point(4,curve), torsion_growth_with_local_data))

##  Conjecture: 

Suppose the curve adds a point of 4-torsion over a quadratic field \( K \), i.e., there is strict growth in $E(\mathbb{Q})[4] \leq E(K)[4] $. Then, if \( p \) is a prime of additive reduction different from 2, \( p \) ramifies in a quadratic field \( L \) (possibly different from \( K \)) such that there is also strict growth in $ E(\mathbb{Q})[4] \leq E(K)[4] $.




In [45]:
def reciprocal_conjecture_add_4_torsion(curve):
    label = curve['lmfdb_label']
    curves_with_fixed_label = list(filter(lambda curve : add_N_torsion_point(4,curve), torsion_growth_with_local_data_grouped_by_label[label]))
    discriminants = [ec['disc_abs'] for ec in curves_with_fixed_label]
    prop = True
    for p, red, kodaira in curve['prime_red_kodaira']:
        if (red == 0) and (p!=2):
            prop = prop and any([discriminant%p == 0 for discriminant in discriminants])        
    return prop

In [46]:
counterexamples = [curve for curve in curves_add_4_torsion if not reciprocal_conjecture_add_4_torsion(curve)]
len(counterexamples)

0

# Torsion after has a new point of order $8$

In [47]:
curves_add_8_torsion = list(filter(lambda curve : add_N_torsion_point(8,curve), torsion_growth_with_local_data))

In [48]:
len(curves_add_8_torsion)

8573

In [49]:
def reciprocal_conjecture_add_8_torsion(curve):
    label = curve['lmfdb_label']
    curves_with_fixed_label = list(filter(lambda curve : add_N_torsion_point(8,curve), torsion_growth_with_local_data_grouped_by_label[label]))
    discriminants = [ec['disc_abs'] for ec in curves_with_fixed_label]
    prop = True
    for p, red, kodaira in curve['prime_red_kodaira']:
        if (red == 0) and (p!=2):
            prop = prop and any([discriminant%p == 0 for discriminant in discriminants])        
    return prop

In [50]:
counterexamples = [curve for curve in curves_add_8_torsion if not reciprocal_conjecture_add_8_torsion(curve)]
len(counterexamples)

0

# Torsion after has a new point of order $16$

In [51]:
curves_add_16_torsion = list(filter(lambda curve : add_N_torsion_point(16,curve), torsion_growth_with_local_data))

In [52]:
len(curves_add_16_torsion)

28

In [53]:
def reciprocal_conjecture_add_16_torsion(curve):
    label = curve['lmfdb_label']
    curves_with_fixed_label = list(filter(lambda curve : add_N_torsion_point(16,curve), torsion_growth_with_local_data_grouped_by_label[label]))
    discriminants = [ec['disc_abs'] for ec in curves_with_fixed_label]
    prop = True
    for p, red, kodaira in curve['prime_red_kodaira']:
        if (red == 0) and (p!=2):
            prop = prop and any([discriminant%p == 0 for discriminant in discriminants])        
    return prop

In [54]:
counterexamples = [curve for curve in curves_add_16_torsion if not reciprocal_conjecture_add_16_torsion(curve)]
len(counterexamples)

0